In [1]:
import pandas as pd
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path = "/content/drive/MyDrive/VK_test/rating.csv"
ratings = pd.read_csv(path)
ratings.head(200)

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
195,2,1270,5.0,2000-11-21 15:36:54
196,2,1327,5.0,2000-11-21 15:34:06
197,2,1356,5.0,2000-11-21 15:29:58
198,2,1544,5.0,2000-11-21 15:35:43


В целом movieId достаточно, но любопытство берёт вверх, изучим что таится за числами

In [66]:
path = "/content/drive/MyDrive/VK_test/movie.csv"
films = pd.read_csv(path)
films.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [67]:
films = films.drop(columns=["genres"])

In [6]:
df = ratings.merge(films, on = "movieId")

In [7]:
df.head()

,userId,movieId,rating,timestamp,title
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995)
1,5,2,3.0,1996-12-25 15:26:09,Jumanji (1995)
2,13,2,3.0,1996-11-27 08:19:02,Jumanji (1995)
3,29,2,3.0,1996-06-23 20:36:14,Jumanji (1995)
4,34,2,3.0,1996-10-28 13:29:44,Jumanji (1995)


In [8]:
df.shape

(20000263, 5)

20 миллионов это конечно круто, но пожалуй, чтобы не умереть от старости ограничемся одним миллионом.

Т.к в датасете гарантируется большое количество оценок для каждого юзера, чтобы это не сломать и не портить метрики околопустыми юзерами (которые могут получиться если случайно семплировать датасет) перед срезом датасета отсортируем его по юзерам.

In [9]:
df = df.sort_values(by="userId")
df = df[:1000000]

На всякий случай проверим не оставили ли мы почти без оценок последнего юзера: 

In [10]:
#id последнего юзера
lastId = df.iloc[-1]["userId"]

#кол-во его рейтингов
df[df["userId"] == lastId].shape[0]

204

*>>Придумайте и обоснуйте способ разбиения данных*

  Основная цель -- построение рекомендации и оценка его качества, то логично будет приблизить обучение модели к настоящей задаче: По истории пользователя предсказать что ему понравится. В реальности мы не обладаем знаниями из будущего, а вкусы могут меняться во времени.

  В таком случае, логично было бы использовать *глобальный срез во времени* -- выбрать точку и считать все до -- историей, а после -- тестовым набором на котором мы будем оценивать наши данные.

  Однако, изучение дат установок оценок показывает что типичное поведение пользователя -- зайти и порасставлять оценки за один день. В результате разбиение по глобальной дате практически не создаст пересечений между тестовой и тренировочной выборкой для конкретного пользователя.
  

In [11]:
df['timestamp'] = pd.to_datetime(df["timestamp"])

In [12]:
#пример пользователя 1 
print(df[df["userId"]==1].shape)
df[df["userId"]==1]["timestamp"].dt.date.unique()

(175, 5)


array([datetime.date(2005, 4, 2), datetime.date(2004, 9, 10)],
      dtype=object)

Можно увидеть что пользователь 1 поставил 175 оценок за 2 дня

Для решения этой проблемы придётся пойти на копромисс и сделать **локальный срез** для каждого пользователя -- отсортируем его оценки по времени и будем считать последние 20% оценок в качестве тестового множества.

Проблема данного подхода в том, что она не совсем реалестична -- т.к "таймлайн" у каждого пользователя может отличаться на *года* мы будем использовать данные из будущего (например что этому фильму user N поствил оценку "5" через пару лет), но в данном случае это не столь уж критично, гораздо лучше чем просто *случайная подвыборка* и решает проблему *глобального среза*.

In [13]:
from tqdm import tqdm

In [79]:
users = df['userId'].unique() #list of all users
movies = df['movieId'].unique() #list of all movies
test = pd.DataFrame(columns=df.columns)
train = pd.DataFrame(columns=df.columns)
test_ratio = 0.2 #fraction of data to be used as test set.
for u in tqdm(users):
    temp = df[df['userId'] == u]
    n = len(temp)
    test_size = int(test_ratio*n)
    temp = temp.sort_values('timestamp').reset_index()
    temp.drop('index', axis=1, inplace=True)

    dummy_test = temp.loc[n-1-test_size :]
    dummy_train = temp.loc[: n-2-test_size]

    test = pd.concat([test, dummy_test])
    train = pd.concat([train, dummy_train])

100%|██████████| 6743/6743 [03:53<00:00, 28.87it/s]


In [ ]:
#df.set_index("timestamp",inplace = True)

In [16]:
!pip3 install implicit

     |████████████████████████████████| 18.5 MB 415 kB/s 


In [87]:
from implicit.als import AlternatingLeastSquares

/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


In [14]:
from scipy.sparse import csr_matrix, save_npz, load_npz, vstack, hstack, lil_matrix

In [ ]:
test

In [84]:
R_df = train.pivot(index = 'userId', columns='movieId', values='rating').fillna(0)
R_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,127166,127196,127212,128488,128510,128520,128594,128842,129030,129707
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_df = test.pivot(index = 'userId', columns='movieId', values='rating').fillna(0)
test_df.head()


In [ ]:
test_df = csr_matrix(test_df)

In [57]:
movies = sorted(train["movieId"].unique())
users = sorted(train["userId"].unique())

In [162]:
R_df.shape

(6743, 12431)

In [85]:
train_sparse = csr_matrix(R_df)

In [88]:
model = AlternatingLeastSquares(factors = 64)

model.fit(train_sparse)

/usr/local/lib/python3.7/dist-packages/implicit/utils.py:27: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  "OpenBLAS detected. Its highly recommend to set the environment variable "


  0%|          | 0/15 [00:00<?, ?it/s]

In [234]:
train_sparse.data

array([3.5, 3.5, 3.5, ..., 3. , 4.5, 3.5])

In [235]:
train_sparse.shape

(6743, 12431)

In [287]:

# recommend items for a user
recommendations, scores = model.recommend(3, train_sparse[3], 30)


In [282]:
true = test[test["userId"]==3]

In [116]:
!pip install ml_metrics

  Created wheel for ml-metrics: filename=ml_metrics-0.1.4-py3-none-any.whl size=7845 sha256=fe6b4ad5bb1c225e71f3ed58be343c0dda66a5cb13c1c19aac55a3528d38299f
  Stored in directory: /root/.cache/pip/wheels/56/41/5b/0c6d42b3604a5c823d8922564c4708f84962fa7f2f4facfa6d
Successfully built ml-metrics


In [91]:
from ml_metrics import mapk, apk

In [316]:
[el - 1 for el in users][-1]

6742

In [89]:
users = [i for i in range(6743)]
recommendations, scores = model.recommend(users, train_sparse[users], 10)

In [92]:
def calculate_ap():
  ap = []
  for userId in tqdm(users):
    true = test[test["userId"]==userId+1]
    true_choice = true.sort_values("rating", ascending = False)["movieId"].values

    ap.append(apk(list(true_choice),list(recommendations[userId]),10))

  return ap

map = calculate_ap()

100%|██████████| 6743/6743 [01:45<00:00, 63.73it/s]


In [93]:
print(np.mean(map))
#0.0012958950082213251
#0.002029838364754511

0.002428434037227636


# Раздел 2

In [16]:
!pip install lightfm

In [68]:
movieIds = [i for i in sorted(films["movieId"])]
userIds = [i for i in sorted(users)]

In [80]:
from lightfm.data import Dataset
# Create your train and test set in the format [[user_id1, item_id1, score1], ..., [user_idn, item_idn, scoren]] 
# Your score can be just 1 for an implicit interaction
# user_id and item_id are integers

data = Dataset()
data.fit(userIds, # list from 0 to n_users 
         movieIds # list from 0 to n_items
        )
train_lfm, weights_matrix = data.build_interactions([tuple(i) for i in train.drop(columns=["timestamp","title"]).values])
test_lfm, weights_matrix = data.build_interactions([tuple(i) for i in test.drop(columns=["timestamp","title"]).values])

In [96]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train_lfm, epochs=10)


AssertionError: ignored

In [82]:
train_precision = precision_at_k(model, train_lfm, k=10).mean()
test_precision = precision_at_k(model, test_lfm, k=10 ).mean()

train_auc = auc_score(model, train_lfm).mean()
test_auc = auc_score(model, test_lfm).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.54, test 0.04.
AUC: train 0.92, test 0.86.


In [69]:
len(movieIds)

27278

In [63]:
movieIds = list(range(27278))

Загрузим мета-информацию

In [368]:
#path = "/content/drive/MyDrive/VK_test/genome_scores.csv"
#tags = pd.read_csv(path)
#tags.head(200)

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675
...,...,...,...
195,1,196,0.05600
196,1,197,0.01975
197,1,198,0.08150
198,1,199,0.06775


In [369]:
tags.shape

(11709768, 3)